<a href="https://colab.research.google.com/github/fvillena/mae2/blob/master/5-annotations-ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Práctico 5: Procesamiento de anotaciones para NER clínico 💻**


----------------------------------

## 📚 **Objetivos de la clase** 📚


El objetivo principal de esta clase es enseñarles las herramientas necesarias para poder procesar archivos de anotaciones en formatos ***Standoff***, los cuáles son ampliamente utilizados en el área del Procesamiento del lenguaje natural.

Específicamente, vamos a abordar los siguientes puntos:

- ¿Qué es el formato Standoff? 
- ¿Qué es el formato ConLL?
- Transformación de Standoff a CoNLL.
- Revisar un ejemplo de la tarea de NER para enfermedades.

# **Motivación** 🙌

**Para empezar, ¿Qué es el procesamiento del lenguaje natural? (NLP)**

NLP es un área de la inteligencia artificial, en la cuál se desarrollan métodos y algoritmos para resolver problemas prácticos (tasks) del lenguaje natural. Ejemplos cotidianos pueden ser: 

1. Clasificar un email en span/no span.
2. Clasificar tweets o películas según su nivel de sentimiento. 
3. Traducir palabras y frases de un idioma a otro.

En particular, alguna de estas tareas de NLP suelen ser modeladas con modelos basados en ***Sequence Labeling***.

&ensp;

**¿Y qué es Sequence Labeling?** 

La idea principal es que dada una secuencia de palabras (oración), los métodos de **sequence labeling** tienen por objetivo asignar una etiqueta a cada palabra de dicha oración. Computacionalmente hablando, dada una lista de tokens esperamos encontrar la mejor secuencia de etiquetas asociadas a esa lista. Pero veamos cómo esta técnica puede ser de utilidad para el área clínica, y para esto es necesario introducir una de las tareas más importantes en NLP.

&ensp;

**Named Entity Recognition (NER)**

NER es una de las tareas más investigadas en el área de NLP y es un ejemplo de un problema de **Sequence Labeling**. Pero antes de definir formalmente de qué trata esta tarea, es necesario definir algunos conceptos claves para poder entenderla fácilmente:

- **Token**: Un token es una secuencia de uno o más caracteres, donde un caracter puede ser una letra, un número o un símbolo. `Ejemplos: ['paciente', 'hta', 'pza', '1.2', '2020'] `

- **Entidad**: No es más que una secuencia de tokens que está asociada a alguna categoría pre-definida. Originalmente se solían utilizar categorías como nombres de personas, organizaciones, ubicaciones, pero actualmente se ha extendido a diferentes dominios. `Ejemplo: Entidad: 'Pablo Baez' -> Categoría: 'Persona'`

- **Límites de una entidad**: Son los índices de los tokens de inicio y fín  de una entidad. `Ejemplo: 'El profesor Pablo Baez' - Entidad: 'Pablo Baez' - Límites: [2, 3]`

- **Tipo de entidad**: Es la categoría predefinida asociada a la entidad.

Por lo tanto, definimos formalmente una entidad como una tupla (conjunto de elementos): $(s, e, t)$, donde $s, t$ son los límites de la entidad (índices de los tokens de inicio y fin, respectivamente) y t corresponde al tipo de entidad o categoría. Ya veremos más ejemplos luego de describir el Dataset.


Dicho esto, podemos imaginar una posible aplicación en el área clínica. Dado un conjunto de diagnósitico sería de gran utilidad reconocer de manera automática las piezas claves de texto que hagan referencia a categorías clínicas, como por ejemplo, enfermedades. Eso es justo lo que haremos 🤙

&ensp;

**Corpus de la Lista de espera**

Trabajaremos con un conjunto de datos reales correspondientes a interconsultas de la lista de espera NO GES en Chile. Si quieren saber más sobre cómo fueron generados los datos pueden revisar el paper publicado hace unos meses atrás en el workshop de EMNLP, una de las conferencias más importantes de NLP: [https://www.aclweb.org/anthology/2020.clinicalnlp-1.32/](https://www.aclweb.org/anthology/2020.clinicalnlp-1.32/).

Este corpus Chileno está constituido originalmente por 12 tipos de entidades:

- **Disease**
- **Body_Part**
- **Medication** 
- **Procedures** 
- **Family_Member**
- **Abbreviation**
- **Laboratory_or_Test_Result**
- **Clinical_Finding** 
- **Diagnostic_Procedure** 
- **Therapeutic_Procedure**
- **Sign_or_Symptom** 
- **Laboratory_Procedure**

Si quieren obtener más información sobre estas entidades pueden consultar la [guía de anotación](https://plncmm.github.io/annodoc/). 


# **Formato Standoff 📄**

Una de las herramientas más populares para generar etiquetado de texto, es el software [**Brat**](https://brat.nlplab.org/). En catedra vieron ejemplos de cómo se realiza este etiquetado y cómo se realiza el cálculo de acuerdo entre distintos anotadores. En la parte práctica, nos centraremos en cómo llegar a un formato adecuado para entrenar un modelo en el área del **[aprendizaje automático.](https://es.wikipedia.org/wiki/Aprendizaje_autom%C3%A1tico)**

Cuando realizamos una anotación en Brat, se genera un archivo con extensión `.ann`, más conocido como el formato **[Standoff.](https://brat.nlplab.org/standoff.html)** Veamos un ejemplo de una interconsulta `referral.txt` y su respectivo archivo de anotaciones `referral.ann`.

In [1]:
# Primero, descargamos ambos archivos desde el repositorio del curso y los situamos una carpeta llamada ann_files.

import os

if not os.path.exists('ann_files/'): 
  os.mkdir('ann_files/')

!wget https://raw.githubusercontent.com/fvillena/mae2/master/data/ann_sample/interconsulta.ann -P ann_files/
!wget https://raw.githubusercontent.com/fvillena/mae2/master/data/ann_sample/interconsulta.txt -P ann_files/

--2021-06-21 03:50:14--  https://raw.githubusercontent.com/fvillena/mae2/master/data/ann_sample/interconsulta.ann
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 272 [text/plain]
Saving to: ‘ann_files/interconsulta.ann’

interconsulta.ann   100%[===================>]     272  --.-KB/s    in 0s      

2021-06-21 03:50:14 (13.3 MB/s) - ‘ann_files/interconsulta.ann’ saved [272/272]

--2021-06-21 03:50:14--  https://raw.githubusercontent.com/fvillena/mae2/master/data/ann_sample/interconsulta.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK

In [2]:
# Primero abrimos el archivo asociado a la interconsulta e imprimimos el contenido.
referral = open('ann_files/interconsulta.txt', 'r', encoding='utf-8').read()
print(referral)

- ANGINA DE PECHO, NO ESPECIFICADA/  - Fundamento Clínico APS: Paciente Hipertensa, con adormecimiento en brazo izquierdo y sensacion de ahogo.


In [3]:
# Luego, abrimos el archivo con la anotación asociada a la interconsulta anterior.
annotation = open('ann_files/interconsulta.ann', 'r', encoding='utf-8').read()
print(annotation)

T1	Disease 2 34	ANGINA DE PECHO, NO ESPECIFICADA
T2	Abbreviation 58 61	APS
T3	Body_Part 12 17	PECHO
T4	Disease 72 82	Hipertensa
T5	Sign_or_Symptom 88 121	adormecimiento en brazo izquierdo
T6	Body_Part 106 121	brazo izquierdo
T7	Sign_or_Symptom 124 142	sensacion de ahogo




El formato Standoff sigue las siguientes reglas:

1.   La primera columna corresponde a un identificador de la **entidad** encontrada.
2.   La segunda columna corresponde al **tipo de entidad**.
3.   La tercera y cuarta columna está asociada a los **límites de la entidad**.
4.   El resto de columnas corresponde al contenido de la entidad en sí, el texto.



El problema es que no es factible entregarle pares de archivos a un algoritmo, sino que deben ser convertidos a un formato conveniente y que sea entendido por python. 

Como el objetivo es crear algoritmos que sean capaces de generar aprendizaje a partir de los datos, lo principal es tener datos etiquetados.

Por ejemplo, si queremos enseñarle a un computador o más bien crear un algoritmo que sea capaz de distinguir si una foto es de un perro o un gato 🐱, lo primero que debemos hacer es pasarle ejemplos al algoritmo con fotos etiquetadas como perro y fotos etiquetadas como gato. 

En nuestro caso es algo similar: La parte de generar datos etiquetados ya está hecha, ahora nos gustaría decirle al algoritmo: "Este token es parte de una enfermedad, este token de una abreviatura, este de una parte del cuerpo, etc", para que genere aprendizaje y cuando lleguen nuevos tokens no vistos antes sea capaz de distinguir a qué categoría pertenece. Para lograr este formato utilizaremos el formato ***ConLL***. 👌

# **Formato ConLL 📄**

Corresponde al formato más utilizado en la tarea de NER y  no es más que un archivo de texto, que cumple las siguientes propiedades:

1. La primera columna contiene los tokens.

2. La segunda columna contiene el tipo de entidad asociado al token de la primera columna.

3. Los tipos de entidades siguen un formato clásico en NER denominado ***IOB2***. Si un tipo de entidad comienza con el prefijo "B-" (Beginning) significa que es el token de inicio de una entidad, si comienza con "I-" (Inside) es un token distinto al de inicio y si un token está asociado a la categoría O (Outside) significa que no pertenece a alguna entidad.

4. Para separar oraciones se utiliza una linea en blanco. Esto es importante ya que al entrenar una red neuronal ustedes pasarán una lista de oraciones como input, más conocidos como batches. 

Aquí va un ejemplo:

```
PACIENTE O
PRESENTA O
FRACTURA B-Disease
CORONARIA I-Disease
COMPLICADA I-Disease
EN O
PIE B-Body_Part
IZQUIERDO I-Body_Part
. O
SE O
REALIZA O
INSTRUMENTACION B-Procedure
INTRACONDUCTO I-Procedure
. O
```

Según nuestra definición tenemos las siguientes tres entidades (enumerando los tokens desde 0): 

- $(2, 4, \text{Disease})$
- $(6, 7, \text{Body Part})$
- $(11, 12, \text{Procedure})$

**Y cómo transformamos de un formato a otro?**

# **Transformación de formatos** 💻 

In [4]:
# Clonamos uno de los cuántos códigos públicos para realizar esta transformación.
!git clone https://github.com/spyysalo/standoff2conll

Cloning into 'standoff2conll'...
remote: Enumerating objects: 101, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 101 (delta 0), reused 1 (delta 0), pack-reused 98
Receiving objects: 100% (101/101), 53.41 KiB | 3.14 MiB/s, done.
Resolving deltas: 100% (58/58), done.


In [5]:
# Ejecutamos el transformador, la idea es tomar todos los pares de anotaciones/interconsultas de la carpeta ann_files y transformarlos al archivo entities.conll
!set PYTHONIOENCODING=utf8
!python standoff2conll/standoff2conll.py ann_files > entities.conll

Eliminate T3	Body_Part 12 17	PECHO due to overlap with T1	Disease 2 34	ANGINA DE PECHO, NO ESPECIFICADA
Eliminate T6	Body_Part 106 121	brazo izquierdo due to overlap with T5	Sign_or_Symptom 88 121	adormecimiento en brazo izquierdo


In [6]:
print(referral)

- ANGINA DE PECHO, NO ESPECIFICADA/  - Fundamento Clínico APS: Paciente Hipertensa, con adormecimiento en brazo izquierdo y sensacion de ahogo.


In [7]:
print(annotation)

T1	Disease 2 34	ANGINA DE PECHO, NO ESPECIFICADA
T2	Abbreviation 58 61	APS
T3	Body_Part 12 17	PECHO
T4	Disease 72 82	Hipertensa
T5	Sign_or_Symptom 88 121	adormecimiento en brazo izquierdo
T6	Body_Part 106 121	brazo izquierdo
T7	Sign_or_Symptom 124 142	sensacion de ahogo




In [8]:
conll_file = open('entities.conll', 'r', encoding='utf-8').read()
print(conll_file)

-	O
ANGINA	B-Disease
DE	I-Disease
PECHO	I-Disease
,	I-Disease
NO	I-Disease
ESPECIFICADA	I-Disease
/	O
-	O
Fundamento	O
Clínico	O
APS	B-Abbreviation
:	O
Paciente	O
Hipertensa	B-Disease
,	O
con	O
adormecimiento	B-Sign_or_Symptom
en	I-Sign_or_Symptom
brazo	I-Sign_or_Symptom
izquierdo	I-Sign_or_Symptom
y	O
sensacion	B-Sign_or_Symptom
de	I-Sign_or_Symptom
ahogo	I-Sign_or_Symptom
.	O




&ensp;

**Problema de las entidades anidadas y el formato ConLL.**

Habrán notado que al ejecutar el programa anterior, el código nos dice que se han eliminado dos entidades: **T3 y T6**. La razón es que son entidades anidadas dentro de otras, la primera está anidada dentro de T1, y la segunda está anidada dentro de T5. Se eliminan ya que si seguimos el formato de CoNLL, no es posible que un token esté asociado a más de un tipo de entidad. Es por esto, que por default nos estamos quedando con la entidad más grande, más conocidas como entidades externas.

Esto es algo muy común en el área de NER cuando nos enfrentamos a un dataset con entidades anidadas, lo que no quiere decir que sea la mejor solución ya que estamos perdiendo mucha información al eliminar las entidades internas. 

Si quieren indagar más sobre este problema, lo pueden buscar como ***Nested Named Entity Recognition***, pero como no es el objetivo, nos quedaremos con que se puede transformar de un problema de entidades anidadas a uno tradicional. Ahora veamos un ejemplo práctico de cómo usar este archivo para entrenar un modelo, donde por simplicidad lo haremos sólo con enfermedades.

# **Ejecutando un modelo** 

In [9]:
# Primero instalamos la librería transformers para poder utilizar un modelo llamado BERT. Este será visto en profundidad más adelante.
!pip install git+https://github.com/huggingface/transformers

# Instalamos una librería que nos permitirá calcular el rendimiento de los modelos.
!pip install datasets seqeval

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-6qeav90_
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-6qeav90_
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 13.9MB/s 
     |████████████████████████████████| 901kB 44.9MB/s 
  Created wheel for transformers: filename=transformers-4.8.0.dev0-cp37-none-any.whl size=2474626 sha256=d157bd7e1d696b78b79ed22e144cb74f4c44728ddbdc8ddd4f6f2cdbbb7485c7
  Stored in directory: /tmp/pip-ephem-wheel-cache-r4x3r3il/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
     |████████████████████████████████| 245kB 14.6MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 
     |████████████████████████████████| 122kB 24.0MB/s 
     |████████████████████████████████| 245kB 28.4MB/s 
  Created wheel for seqe

In [10]:
# Instalamos la librería de dataset que nos entrega hugging face y que nos simplifica la vida al momento de procesar archivos con formato CoNLL.
!pip install datasets

In [11]:
# Un par de librerías necesarias para ejecutar el modelo.
import transformers
import datasets
import numpy as np
import torch

In [12]:
# Cargamos el modelo de BERT en Español creado por investigadores del departamento de Computación de la Universidad de Chile. Para mayor información visitar este sitio: https://github.com/dccuchile/beto
bert_path = 'dccuchile/bert-base-spanish-wwm-cased'

# Cargamos el dataset de enfermedades. Visitar la siguiente página para ver los CoNLL creados: https://huggingface.co/datasets/mrojas/disease/tree/main/data.
dataset = datasets.load_dataset("mrojas/disease")

Dataset disease downloaded and prepared to /root/.cache/huggingface/datasets/disease/disease/1.0.0/f60670e31a5bb8dff8979d0581f1eb157adb8bf2e08b6d716198558f1c151419. Subsequent calls will reuse this data.


In [13]:
# Particiones y tamaño de cada una.
dataset.shape

{'test': (992, 2), 'train': (8025, 2), 'validation': (891, 2)}

In [14]:
# Nombres de las columnas utilizadas 
dataset.column_names

{'test': ['ner_tags', 'tokens'],
 'train': ['ner_tags', 'tokens'],
 'validation': ['ner_tags', 'tokens']}

In [15]:
# Revisamos los diferentes tipos de entidades que hay en el dataset
label_list = dataset["train"].features["ner_tags"].feature.names
label_list

['O', 'B-Disease', 'I-Disease']

In [16]:
# Printeamos un ejemplo. Aquí se puede ver que por cada token tenemos un tag asociado, que es justamente lo que habíamos explicado.
dataset["train"][0]

{'ner_tags': [1,
  2,
  2,
  0,
  1,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  2,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'tokens': ['K08',
  '.',
  '1',
  '-',
  'PERDIDA',
  'DE',
  'DIENTES',
  'DEBIDA',
  'A',
  'ACCIDENTE',
  ',',
  'EXTRACCION',
  'O',
  'ENF',
  '.',
  'PERIODONTAL',
  'LOCAL',
  '/',
  'Se',
  'solicita',
  'Protesis',
  'Parcial',
  'superior',
  'e',
  'inferior']}

In [17]:
# Se utiliza un tokenizador especial de BERT para obtener representaciones numéricas de cada token.
tokenizer = transformers.BertTokenizerFast.from_pretrained(
    bert_path)

In [18]:
# Realizamos la tokenización obteniendo una representación numérica de los ejemplos.
label_all_tokens = False
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

In [19]:
tokenized_dataset["train"][0]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'input_ids': [4,
  991,
  10724,
  1009,
  1094,
  1149,
  11931,
  13617,
  8739,
  1982,
  20606,
  22739,
  1982,
  6518,
  8739,
  1070,
  4777,
  2277,
  8039,
  30969,
  6356,
  1017,
  1044,
  985,
  18590,
  10540,
  7232,
  30969,
  1206,
  4202,
  30999,
  1009,
  11931,
  7232,
  8375,
  30969,
  5409,
  30970,
  12755,
  6399,
  30970,
  972,
  1264,
  13170,
  15174,
  4143,
  1561,
  1418,
  3471,
  1007,
  5413,
  5],
 'labels': [-100,
  1,
  -100,
  2,
  2,
  0,
  1,
  -100,
  -100,
  2,
  2,
  -100,
  2,
  -100,
  -100,
  2,
  2,
  -100,
  -100,
  -100,
  -100,
  2,
  2,
  -100,
  -100,
  -100,
  -100,
  -100,
  2,
  2,
  -100,
  2,
  2,
  -100,
  -100,
  -100,
  -100,
  -100,
  2,
  -100,
  -1

In [20]:
# Definimos los hiperparámetros del modelo y la métrica
batch_size = 16
args = transformers.TrainingArguments(
    f"test-ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

metric = datasets.load_metric("seqeval")

In [21]:
data_collator = transformers.DataCollatorForTokenClassification(tokenizer)

In [22]:
# Cargamos el modelo de BERT en Español mencionado anteriormente.
model = transformers.AutoModelForTokenClassification.from_pretrained(bert_path, num_labels=len(label_list))

Some weights of the model checkpoint at dccuchile/bert-base-spanish-wwm-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at dccuchile/bert-base

In [23]:
# Entrenamos el modelo.

trainer = transformers.Trainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.224200,0.172463,0.689861,0.711795,0.700656,0.942511
2,0.121700,0.172981,0.741658,0.706667,0.723739,0.946191
3,0.084600,0.193190,0.739754,0.740513,0.740133,0.947880


TrainOutput(global_step=1506, training_loss=0.14336555586709762, metrics={'train_runtime': 206.6585, 'train_samples_per_second': 116.497, 'train_steps_per_second': 7.287, 'total_flos': 1768736987679870.0, 'train_loss': 0.14336555586709762, 'epoch': 3.0})

In [24]:
# Vemos el rendimiento del modelo en el conjunto de test

predictions, labels, _ = trainer.predict(tokenized_dataset["test"])
predictions = np.argmax(predictions, axis=2)


true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'Disease': {'f1': 0.7625113739763419,
  'number': 1077,
  'precision': 0.7475468331846565,
  'recall': 0.7780872794800371},
 'overall_accuracy': 0.9573938902406056,
 'overall_f1': 0.7625113739763419,
 'overall_precision': 0.7475468331846565,
 'overall_recall': 0.7780872794800371}

In [25]:
# Inferencia de un ejemplo

sequence = "HTA DM CA COLON OPERADO ANEMIA TROMBOSIS HPB MARCAPASOS ULTIMO CONTROL DE TELEMETRIA ABRIL15 HISTOGRAMA SIN EVENTOS MCP CON BUEN SENSADO Y CAPTURA, TVP VENA AXILAR IZQUIERDA EN TACO LE DETECTARON GLAUCOMA EN TTO"

tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(sequence)))
inputs = tokenizer.encode(sequence, return_tensors="pt").to("cuda")
outputs = model(inputs)[0]
predictions = torch.argmax(outputs, dim=2)
for token, prediction in zip(tokens, predictions[0].tolist()):
  print(token, label_list[prediction])

[CLS] O
H B-Disease
##TA B-Disease
D B-Disease
##M B-Disease
CA B-Disease
COL I-Disease
##ON I-Disease
OP O
##ERA O
##DO O
ANE B-Disease
##MI B-Disease
##A I-Disease
TR B-Disease
##OM I-Disease
##BO I-Disease
##SI I-Disease
##S I-Disease
H B-Disease
##P B-Disease
##B O
MAR O
##CA O
##PA O
##SO O
##S O
U O
##L O
##TI O
##MO O
CONTR O
##OL O
DE O
TE O
##LE O
##ME O
##TR O
##IA O
AB O
##RI O
##L O
##15 O
H B-Disease
##IST B-Disease
##OG O
##RA O
##MA O
SIN O
EV O
##ENTOS O
MC O
##P O
CON O
BU O
##EN O
SE O
##N O
##SA O
##DO O
Y O
CAP O
##TURA O
, O
TV O
##P O
VE O
##NA O
A O
##X O
##IL O
##AR O
I O
##Z O
##QU O
##IER O
##DA O
EN O
TA O
##CO O
LE O
DE O
##TE O
##C O
##TA O
##RO O
##N O
G B-Disease
##LA B-Disease
##UC O
##OM O
##A O
EN O
T O
##TO O
[SEP] O
